In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist=input_data.read_data_sets("../data/MNIST_data/",one_hot=True)

Extracting ../data/MNIST_data/train-images-idx3-ubyte.gz
Extracting ../data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../data/MNIST_data/t10k-labels-idx1-ubyte.gz


In [17]:
# 定义超参数和其它常量
## Data specific constants
n_input=784 #img shape: 28*28
n_classes=10

## hyperparameters
max_epoches=10000
learning_rate=0.5
batch_size=10
seed = 0
n_hidden=30

In [5]:
# 激活函数的导数
def sigmprime(x):
    return tf.multiply(tf.sigmoid(x), 
                       tf.subtract(tf.constant(1.0), tf.sigmoid(x)))

In [6]:
# 训练数据占位符
x=tf.placeholder(tf.float32,shape=[None, n_input], name='X')
y=tf.placeholder(tf.float32,shape=[None, n_classes], name='Y')


In [7]:
# 创建模型
def multilayer_perceptron(x, weights, biases):
    
    h_layer_1=tf.add(tf.matmul(x, weights['h1']), biases['h1'])
    out_layer_1=tf.sigmoid(h_layer_1)
    h_out=tf.matmul(out_layer_1,weights['out']) + biases['out']
    return tf.sigmoid(h_out), h_out, out_layer_1, h_layer_1

In [9]:
# 定义权重变量
weights={
    'h1':tf.Variable(tf.random_normal([n_input, n_hidden], seed=seed)),
    'out':tf.Variable(tf.random_normal([n_hidden, n_classes], seed=seed))
}

biases={
    'h1':tf.Variable(tf.random_normal([1, n_hidden], seed=seed)),
    'out':tf.Variable(tf.random_normal([1, n_classes], seed=seed))
}

In [11]:
# 创建模型（计算图）
## Forward 
y_hat, h_2,o_1,h_1=multilayer_perceptron(x, weights=weights,biases=biases)
## Error
err=y_hat-y
# Backword
delta_2=tf.multiply(err, sigmprime(h_2))
delta_w_2=tf.matmul(tf.transpose(o_1), delta_2)
wtd_error=tf.matmul(delta_2,tf.transpose(weights['out']))
delta_1=tf.multiply(wtd_error, sigmprime(h_1))
delta_w_1=tf.matmul(tf.transpose(x), delta_1)
eta=tf.constant(learning_rate)
## update weights
step=[
    tf.assign(weights['h1'],
              tf.subtract(weights['h1'],tf.multiply(eta,delta_w_1))),
    tf.assign(biases['h1'],
             tf.subtract(biases['h1'], tf.multiply(eta,tf.reduce_mean(delta_1,axis=0)))),
    tf.assign(weights['out'],
              tf.subtract(weights['out'],tf.multiply(eta,delta_w_2))),
    tf.assign(biases['out'],
             tf.subtract(biases['out'], tf.multiply(eta,tf.reduce_mean(delta_2,axis=0))))
]

In [14]:
# 准确率
acct_mat=tf.equal(tf.argmax(y_hat, 1), tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(acct_mat, tf.float32))


In [19]:
# run
init_op=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init_op)
    for epoch in range(max_epoches):
        batch_xs, batch_ys=mnist.train.next_batch(batch_size)
        sess.run(step, feed_dict={x:batch_xs,y:batch_ys})
        if epoch%100==0:
            acc_test=sess.run(accuracy,
                              feed_dict={x:mnist.test.images, y:mnist.test.labels})
            acc_train=sess.run(accuracy,
                              feed_dict={x:mnist.train.images,y:mnist.train.labels})
            
            print('Epoch: {0} Accuracy Train%: {1} Accuracy Test%: {2}'.
                  format(epoch, acc_train*100, acc_test*100))

Epoch: 0 Accuracy Train%: 12.894545495510101 Accuracy Test%: 13.3200004696846
Epoch: 100 Accuracy Train%: 39.156362414360046 Accuracy Test%: 38.94000053405762
Epoch: 200 Accuracy Train%: 49.78181719779968 Accuracy Test%: 49.68000054359436
Epoch: 300 Accuracy Train%: 61.60181760787964 Accuracy Test%: 62.08000183105469
Epoch: 400 Accuracy Train%: 74.83999729156494 Accuracy Test%: 76.08000040054321
Epoch: 500 Accuracy Train%: 76.74182057380676 Accuracy Test%: 77.53999829292297
Epoch: 600 Accuracy Train%: 79.93090748786926 Accuracy Test%: 80.36999702453613
Epoch: 700 Accuracy Train%: 81.58727288246155 Accuracy Test%: 82.41999745368958
Epoch: 800 Accuracy Train%: 83.74181985855103 Accuracy Test%: 83.67000222206116
Epoch: 900 Accuracy Train%: 83.24363827705383 Accuracy Test%: 83.24000239372253
Epoch: 1000 Accuracy Train%: 84.3745470046997 Accuracy Test%: 84.74000096321106
Epoch: 1100 Accuracy Train%: 84.5054566860199 Accuracy Test%: 84.71999764442444
Epoch: 1200 Accuracy Train%: 85.325455665